In [2]:
import json
from collections import defaultdict
from sklearn import linear_model, model_selection
import numpy as np
import random
import gzip
import dateutil.parser
import math
import mido
import pretty_midi
import joblib
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec
import collections
import os
import tarfile
import tables
import hdf5_getters as GETTERS
from midi_methods import *

%store -r

In [1]:
"""
# UNCOMMENT AND RUN THIS BLOCK OF CODE IF YOU HAVE NOT EXTRACTED MIDI FILES
# FROM GZ FILE

# Download following file at http://hog.ee.columbia.edu/craffel/lmd/lmd_aligned.tar.gz
# Put it in the resources folder!!!

f = tarfile.open("resources/lmd_matched.tar.gz")
f.extractall()
f.close

# Download following file at http://hog.ee.columbia.edu/craffel/lmd/lmd_matched_h5.tar.gz
# Put it in the resources folder!!!

f = tarfile.open("resources/lmd_matched_h5.tar.gz")
f.extractall()
f.close
"""



'\n# UNCOMMENT AND RUN THIS BLOCK OF CODE IF YOU HAVE NOT EXTRACTED MIDI FILES\n# FROM GZ FILE\n\n# Download following file at http://hog.ee.columbia.edu/craffel/lmd/lmd_aligned.tar.gz\n# Put it in the resources folder!!!\n\nf = tarfile.open("resources/lmd_matched.tar.gz")\nf.extractall()\nf.close\n\n# Download following file at http://hog.ee.columbia.edu/craffel/lmd/lmd_matched_h5.tar.gz\n# Put it in the resources folder!!!\n\nf = tarfile.open("resources/lmd_matched_h5.tar.gz")\nf.extractall()\nf.close\n'

In [37]:
# https://github.com/craffel/pretty-midi/blob/main/Tutorial.ipynb

# Some song that was extracted
test_song_path = 'lmd_matched/A/A/A/TRAAAGR128F425B14B/1d9d16a9da90c090809c153754823c2b.mid'

song_pm = pretty_midi.PrettyMIDI(test_song_path)
all_instruments = song_pm.instruments
# Instrument.program to get program number
all_instruments[:5]


[Instrument(program=62, is_drum=False, name="main synth          "),
 Instrument(program=62, is_drum=False, name="echo synth          "),
 Instrument(program=48, is_drum=False, name="main strings h      "),
 Instrument(program=48, is_drum=False, name="main strings m      "),
 Instrument(program=48, is_drum=False, name="main strings l      ")]

In [5]:
# FIrst instrument listed is main synth. Name isnt as important as program number
# as name can be assigned to anything.
main_synth = all_instruments[0]
main_synth

Instrument(program=62, is_drum=False, name="main synth          ")

In [38]:
# Get specific notes played for each instrument
notes_main_synth = main_synth.notes
notes_main_synth[0], notes_main_synth[-1] 

(Note(start=0.018229, end=0.184896, pitch=69, velocity=100),
 Note(start=215.851562, end=215.989583, pitch=71, velocity=100))

In [7]:
# Respective song's hdf5 file

test_hdf5_path = 'lmd_matched_h5\A\A\A\TRAAAGR128F425B14B.h5'
h5 = GETTERS.open_h5_file_read(test_hdf5_path)
# Get song metadata
h5.root.metadata

/metadata (Group) 'metadata about the song'
  children := ['artist_terms' (EArray), 'artist_terms_freq' (EArray), 'artist_terms_weight' (EArray), 'similar_artists' (EArray), 'songs' (Table)]

In [36]:
# Get song title
h5.root.metadata.songs.cols.title[:]

array([b'Into The Nightlife'], dtype='|S1024')

In [ ]:
# Get album

h5.root.metadata.songs.cols.release[:]

In [ ]:
# Get song id. I don't know what the id is referring to (just that dataset or 
# also a foreign key to another datset)

h5.root.metadata.songs.cols.song_id[:]

In [ ]:
# Get song artist

h5.root.metadata.songs.cols.artist_name[:]

In [34]:
# Getting genre of a song. If its empty, that means it has no genre assigned in 
# the dataset, but may still have a genre
h5.root.metadata.songs.cols.genre[:]

array([b''], dtype='|S1024')

In [10]:
#https://github.com/tbertinmahieux/MSongsDB/blob/master/PythonSrc/hdf5_getters.py